**A* Search**

Implementation with Visualization of (6x10) matrix at each step. Shows how search explores and finds the goal get the path. 

**Note:** If you want just the final answer then its at the last cell

In [1]:
import pandas as pd
import numpy as np
from IPython.display import Markdown, display
import heapdict

# creating dictionary that stores info abour each cell of the cliff walker maze as per the question
maze_dict=dict()

start_position = (6,1)
goal_position = (6,10)
fringe_a_star=heapdict.heapdict() # heap_dictionary that works like a priority queue for implementing A*
path_a_star=[] # returned path for A*

# backtracking_dict dictionary will be later used to back track the path of traversal that actually led to finding goal
backtracking_dict={start_position:[]}

#DeathZone in red as per question
#Shallow region in white as per question
#Deep region in green as per question
def init_cliff_walker(start_pos, goal_pos):
  for i in range(1,7):
    for j in range(1,11):
      if i==6 and (j<=9 and j>=2):
        maze_dict[(i,j)] = [" ","Deathzone","unvisited"]
      elif (i>=3 and i<=5) and ((j<=4 and j>=2) or (j<=8 and j>=6)):
        maze_dict[(i,j)] = [" ","Deep","unvisited"]
      else:
        maze_dict[(i,j)] = [" ","Shallow","unvisited"]
  maze_dict[start_pos][0] = "START"
  maze_dict[goal_pos][0] = "GOAL"

def print_cliff_walker():
  row_lst = []
  for row_num in range(1,7):
    lst=[]
    for col_num in range(1,11):
      if (row_num,col_num) == start_position or (row_num,col_num) == goal_position or ((row_num,col_num) in path_a_star):
        lst.append(str(maze_dict[(row_num,col_num)][0]))
      elif maze_dict[(row_num,col_num)][2] == "unvisited":
        lst.append(5*'_')
      elif maze_dict[(row_num,col_num)][2] == "visited" and maze_dict[(row_num,col_num)][0] != " ":
        lst.append("\u03BF")
      elif maze_dict[(row_num,col_num)][2] == "visited":
        lst.append("\u03BF")
    row_lst.append(lst)
  
  idx_lst=[str(i) for i in range(1,7)]
  col_lst=[str(i) for i in range(1,11)]

  data_arr = np.array(row_lst)
  df = pd.DataFrame(data_arr, index=idx_lst, columns=col_lst)
  df = df.style.apply(style_specific_cell, axis=None)
  return df

def style_specific_cell(x):

    font_typ = 'font-weight: bold; text-align: center;'
    bg_color = ''
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    for key in maze_dict.keys():
      df1.iloc[key[0]-1, key[1]-1] = font_typ
      if maze_dict[key][1] == "Deep":
        bg_color = 'background-color: lightgreen;'
      elif maze_dict[key][1] == "Deathzone":
        bg_color = 'background-color: red;'
      elif maze_dict[key][1] == "Shallow":
        bg_color = 'background-color: white;'
      df1.iloc[key[0]-1, key[1]-1] += bg_color

      if key in path_a_star:
        df1.iloc[key[0]-1, key[1]-1] += 'color: blue;'
      else:
        df1.iloc[key[0]-1, key[1]-1] += 'color: black;'
        
    return df1

# we are using Manhattan distancce as heuristic function to select cells closest to GOAL
def get_heuristic(a,b):
  return abs(a[0]-b[0])+abs(a[1]-b[1])

# For A* search, we combine costs of g(n) and heuristic function h(n)
def get_cell_expense(cell_type):
  if cell_type == "Deep":
    return 5
  elif cell_type == "Shallow":
    return 1

# checks for valid neighbors/children to explore as per environment conditions. 
# Here valid means maze boundary condtions to be in 6x10 matrix, not encountering red deathzone and not revisiting already visited node
def get_valid_neighbors(curr_pos):
  neighbors = [(curr_pos[0]-1,curr_pos[1]), (curr_pos[0],curr_pos[1]-1), (curr_pos[0],curr_pos[1]+1), (curr_pos[0]+1,curr_pos[1])]
  valid=[]
  for i in neighbors:
    if(i[0]<7 and i[0]>0) and (i[1]>0 and i[1]<11) and maze_dict[i][2] == "unvisited" and maze_dict[i][1]!="Deathzone":
      valid.append(i)
  return valid

  
init_cliff_walker(start_position, goal_position)
print("Initiating empty maze for A*")
display(print_cliff_walker())


def a_star():

  print("Fringe => "+ str(list(fringe_a_star.items()))) # Printing the fringe in format (location, f(n) costing)
  curr = fringe_a_star.popitem()[0]
  
  # ignores visited positions to avoid redundant operation
  while maze_dict[curr][2] == "visited":
    curr = fringe_a_star.popitem()[0] # popped item is (location, cost)
  
  maze_dict[curr][2] = "visited"
  print("popped "+str(curr)+" and tracked below")
  display(print_cliff_walker())
  if maze_dict[curr][0]=="GOAL":
    path_a_star.append(curr)
    backtracking_dict[tuple()]= [curr]
    return True
  
  #collecting valid neighbors to expand the A* tree
  children = get_valid_neighbors(curr)
  # adding new children/neighbors to fringe (priority queue)
  for child in children:
    h = get_heuristic(child, goal_position)
    g = get_cell_expense(maze_dict[child][1])
    f = g + h
    # A* depends on two functions to decide its next step: f(n) = g(n) + h(n)
    fringe_a_star[child] = f

  #updating backtracking dictionary
  for child in children:
    if child not in backtracking_dict:
      backtracking_dict[child] = [curr]
    # if child in backtracking_dict:
    #   backtracking_dict[child].append(curr)
    # else:
    #   backtracking_dict[child] = [curr]
  
  # continues recursive call till fringe empty or until GOAL not found
  while (fringe_a_star.items())!=[]:  
    result = a_star()
    if result == True:
      return True  
  return False


#adding root/start node to a_star-fringe to start A*
fringe_a_star[start_position] = get_cell_expense(maze_dict[start_position][1]) + get_heuristic(start_position, goal_position) # f(n) = g(n) + h(n)
if a_star():
  print("A* found GOAL")
else:
  print("A* failed to find GOAL")

#using backtracking to get path
temp_key=(6,10)
while backtracking_dict[temp_key] != []:
  ans = backtracking_dict[temp_key][0]
  path_a_star.append(ans)
  temp_key = ans
path_a_star = [path_a_star[x] for x in range( len(path_a_star)-1,-1,-1)] # reversing list to go from START to GOAL
print("A* path backtracked START to GOAL =>"+str(path_a_star))

a_star_path_cost=0 # gives path cost sum
for i in path_a_star:
  a_star_path_cost += get_cell_expense(maze_dict[i][1])
  if i == start_position or i == goal_position:
    maze_dict[i][0]+= "="+str(a_star_path_cost)
  else:
    maze_dict[i][0]=a_star_path_cost  

print("A* Path Cost = "+str(a_star_path_cost))
print()
print("Output of explored areas with output cost at each step higlighted in blue color:")
display(print_cliff_walker())

Initiating empty maze for A*


,1,2,3,4,5,6,7,8,9,10
1,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
2,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
3,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
4,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
5,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
6,START,_____,_____,_____,_____,_____,_____,_____,_____,GOAL


Fringe => [((6, 1), 10)]
popped (6, 1) and tracked below


,1,2,3,4,5,6,7,8,9,10
1,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
2,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
3,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
4,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
5,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
6,START,_____,_____,_____,_____,_____,_____,_____,_____,GOAL


Fringe => [((5, 1), 11)]
popped (5, 1) and tracked below


...

,1,2,3,4,5,6,7,8,9,10
1,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
2,ο,ο,ο,ο,ο,ο,ο,ο,ο,_____
3,ο,_____,_____,_____,ο,_____,_____,_____,ο,_____
4,ο,_____,_____,_____,ο,_____,_____,_____,ο,_____
5,ο,_____,_____,_____,ο,_____,_____,_____,ο,ο
6,START,_____,_____,_____,_____,_____,_____,_____,_____,GOAL


Fringe => [((5, 2), 14), ((4, 2), 15), ((1, 1), 15), ((1, 2), 14), ((3, 2), 16), ((1, 3), 13), ((3, 3), 15), ((1, 4), 12), ((1, 5), 11), ((3, 4), 14), ((4, 4), 13), ((4, 6), 11), ((5, 4), 12), ((5, 6), 10), ((1, 6), 10), ((3, 6), 12), ((1, 7), 9), ((3, 7), 11), ((1, 8), 8), ((1, 9), 7), ((2, 10), 5), ((3, 8), 10), ((3, 10), 4), ((4, 8), 9), ((5, 8), 8), ((4, 10), 3), ((6, 10), 1)]
popped (6, 10) and tracked below


,1,2,3,4,5,6,7,8,9,10
1,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
2,ο,ο,ο,ο,ο,ο,ο,ο,ο,_____
3,ο,_____,_____,_____,ο,_____,_____,_____,ο,_____
4,ο,_____,_____,_____,ο,_____,_____,_____,ο,_____
5,ο,_____,_____,_____,ο,_____,_____,_____,ο,ο
6,START,_____,_____,_____,_____,_____,_____,_____,_____,GOAL


A* found GOAL
A* path backtracked START to GOAL =>[(6, 1), (5, 1), (4, 1), (3, 1), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 9), (4, 9), (5, 9), (5, 10), (6, 10)]
A* Path Cost = 18

Output of explored areas with output cost at each step higlighted in blue color:


,1,2,3,4,5,6,7,8,9,10
1,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
2,5,6,7,8,9,10,11,12,13,_____
3,4,_____,_____,_____,ο,_____,_____,_____,14,_____
4,3,_____,_____,_____,ο,_____,_____,_____,15,_____
5,2,_____,_____,_____,ο,_____,_____,_____,16,17
6,START=1,_____,_____,_____,_____,_____,_____,_____,_____,GOAL=18


**Final answer of A* below**

In [2]:
print("A* path backtracked START to GOAL =>"+str(path_a_star))
print("A* Path Cost = "+str(a_star_path_cost))
print()
print("Output of explored areas with output cost at each step higlighted in blue color:")
display(print_cliff_walker())

A* path backtracked START to GOAL =>[(6, 1), (5, 1), (4, 1), (3, 1), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 9), (4, 9), (5, 9), (5, 10), (6, 10)]
A* Path Cost = 18

Output of explored areas with output cost at each step higlighted in blue color:


,1,2,3,4,5,6,7,8,9,10
1,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
2,5,6,7,8,9,10,11,12,13,_____
3,4,_____,_____,_____,ο,_____,_____,_____,14,_____
4,3,_____,_____,_____,ο,_____,_____,_____,15,_____
5,2,_____,_____,_____,ο,_____,_____,_____,16,17
6,START=1,_____,_____,_____,_____,_____,_____,_____,_____,GOAL=18
